based on this:
https://python.langchain.com/docs/tutorials/local_rag/

In [1]:
# # Document loading, retrieval methods and text splitting
# !pip install -qU langchain langchain_community

# # Local vector store via Chroma
# !pip install -qU langchain_chroma

# # Local inference and embeddings via Ollama
# !pip install -qU langchain_ollama

# # Web Loader
# ! pip install -qU beautifulsoup4

### Add the project root path to the python path

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.abspath(os.curdir), '..'))

## Load documents

In [3]:
from postgres_connection import get_postgress_data, create_psql_connection
from sql_files import sql_files
import pandas as pd

In [4]:
COUNTRY_NAME = 'MLI Mali'
SPOKEN_LANGUAGE = "French"

df = get_postgress_data(sql_files['get_docs_per_country'].replace("%country_name%", COUNTRY_NAME))

/home/andrei/Ferdi_LLM/misc_notebooks/../postgres_connection.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
from langchain_community.document_loaders import DataFrameLoader

dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

loader = DataFrameLoader(df, page_content_column="content")
data = loader.load()

## Load documents

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

CHUNK_SIZE = 1024

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=int(CHUNK_SIZE/10))
all_splits = text_splitter.split_documents(data)

## Create vector store

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings

EMBEDDING_MODEL_NAME = "dunzhang/stella_en_1.5B_v5"

local_embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

/tmp/ipykernel_2778182/2830179981.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  local_embeddings = HuggingFaceEmbeddings(
You try to use a model that was created with version 3.0.1, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/home/andrei/anaconda3/envs/Ferdi_env_2/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been a

In [15]:
from langchain.vectorstores import FAISS

# vectorstore = FAISS.from_documents(
#     all_splits, 
#     embedding=local_embeddings
#     )

# vectorstore.save_local(folder_path=f"./FAISS_MLI Mali_{CHUNK_SIZE}")

vectorstore = FAISS.load_local(
    "/home/andrei/Ferdi_LLM/misc_notebooks/lanchain_storage_MLI Mali", 
    embeddings=local_embeddings, 
    allow_dangerous_deserialization=True)

In [16]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.1",
    temperature=0,
)

### Naive retriever approach

#### Retriever

In [51]:
# naive_retriever = vectorstore.as_retriever(search_kwargs={ "k" : 10})
naive_retriever = vectorstore.as_retriever(search_kwargs={"score_threshold": 0.5}, search_type="similarity_score_threshold")
# naive_retriever = vectorstore.as_retriever(search_type="mmr")


#### Augument & Generate

In [52]:
from langchain_core.prompts import ChatPromptTemplate

# Augmented
TEMPLATE = """\
You are an intelligent assistant. 
Based on the knowledge provided below, retrieve the relevant information, summarize it and then answer the question.
If you don't know the answer, reply with "I don't know."

Consider these definitions:
'_Gen_CGI': 'Code général des impôts', '_Gen_IT': 'Impôt sur le revenu', '_Gen_VAT': 'Taxe sur la valeur ajoutée', '_Gen_NHI': 'Assurance maladie nationale', '_Gen_LF': 'Loi de finances', '_Invest_CI': 'Code des investissement', '_Min_CM': 'Code minier', '_Pétrol_CP': 'Code pétrolier', '_Gaz_CG': 'Code gazier', '_Forest_CF': 'Code forestier'

Question:
{question}

Here is the knowledge base:
{knowledge}
The above is the knowledge base.
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)


# Generation
chat_model = model

#### RAG put together

In [53]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

setup_and_retrieval = RunnableParallel({"question": RunnablePassthrough(), "knowledge": naive_retriever })
output_parser = StrOutputParser()


naive_retrieval_chain = setup_and_retrieval \
                        | rag_prompt \
                        | chat_model \
                        | output_parser

In [54]:
definitions = {'_Gen_CGI': 'Code général des impôts', '_Gen_IT': 'Impôt sur le revenu', '_Gen_VAT': 'Taxe sur la valeur ajoutée', '_Gen_NHI': 'Assurance maladie nationale', '_Gen_LF': 'Loi de finances', '_Invest_CI': 'Code des investissement', '_Min_CM': 'Code minier', '_Pétrol_CP': 'Code pétrolier', '_Gaz_CG': 'Code gazier', '_Forest_CF': 'Code forestier'}
doc_name = "Loi n°1996-030 (12.06.1996) Office malien de l_habitat 1996 (SGG)"

response = naive_retrieval_chain.invoke(
    f"""Consider the contents of the document with this title '{doc_name}'. In which of these categories : _Gen_CGI, _Gen_IT, _Gen_VAT, _Gen_NHI, _Gen_LF, _Invest_CI, _Min_CM, _Pétrol_CP, _Gaz_CG, _Forest_CF, is it part of ? Answer with a valid category. In the final answer, keep the category only, even if it's none.
    """
    )

/home/andrei/anaconda3/envs/Ferdi_env_2/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1042: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'title': 'img_JO 1996 n°011 (15.06.1996) (SGG)', 'country': 'MLI Mali', 'start_index': 10221}, page_content="4 juin 1996 arrêté N°96-0907/MFC-SG portant nomination d'un Agent Comptable à l'Office du Niger....p645\n\nMINISTERE DE L'EMPLOI DE LA FONCTION PUBLIQUE ET DU TRAVAIL\n\n14 mai 1996 arrêté N°96-0772/MEFPT-SG portant radiation....p645\n\nMINISTERE DU DEVELOPPEMENT RURAL ET DE L'ENVIRONNEMENT\n\n17 mai 1996 arrêté N°96-0778/MDRE-SG portant composition et fonctionnement des commissions régionales chargées de définir les critères relatifs à la fixation des quotas d'exploitation forestière....p645\n\n23 mai 1996 arrêté N°96-0818/MDRE-SG portant nomination du Directeur Adjoint à l'Office de la Haute Vallée du Niger....p645\n\narrêté N°96-0819/MDRE-SG portant nomination d'un chef de Division à la D

In [55]:
print(response)

I don't know.


### Self Query Retriever approach

In [56]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers.self_query.chroma import ChromaTranslator



metadata_field_info = [
    AttributeInfo(
        name="Document_Title",
        description="The title of the document",
        type="string",
    ),
    AttributeInfo(
        name="Country",
        description="The country of the document",
        type="string",
    ),
    AttributeInfo(
        name="Law_Type",
        description="The law type of the document",
        type="string",
    )
]

In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

doc_name = "Décret n°1996-179 (19.06.1996) Application Office malien de l_habitat 1996 (SGG)"

chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = f"Based on the content and title of this text {df[df.title == doc_name]['content'].iloc[0]}, what's the year it was issued? Keep the year only."

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

"Il semble que vous avez fourni une liste de décrets du Mali publiés en 2006, plutôt qu'un texte spécifique à analyser. Je vais donc essayer de répondre à votre question générale sur l'importance des décrets dans le système juridique malien.\n\nLes décrets sont des actes normatifs pris par les organes exécutif du Mali (le Président de la République et le Premier Ministre) pour mettre en œuvre des lois ou règlements. Ils ont une portée spécifique et visent à régir des aspects particuliers de l'activité publique.\n\nDans le contexte de votre question, les décrets peuvent avoir plusieurs fonctions :\n\n1. **Interprétation des lois** : Les décrets peuvent être utilisés pour interpréter des dispositions de loi, en leur donnant une portée plus précise ou en les adaptant à des situations spécifiques.\n2. **Mise en œuvre de la loi** : Les décrets peuvent être pris pour mettre en œuvre les dispositions d'une loi, en fixant des règles ou des procédures nécessaires à sa mise en pratique.\n3. **Cr

In [49]:
# df[df.title == doc_name]['content'].iloc[0]
docs

[Document(metadata={'title': 'Décret n°1996-179 (19.06.1996) Application Office malien de l_habitat 1996 (SGG)', 'country': 'MLI Mali', 'start_index': 0}, page_content='Décret N°96-179/P-RM fixant l\x92organisation et les modalités de fonctionnement de l\x92office malien de l\x92habitat.\n\nLe Président de la République,\n\nVu la Constitution,\n\nVu l\x92Ordonnance N°46 Bis/PGP du 16 novembre 1961 portant règlement financier en République du Mali ;\n\nVu l\x92Ordonnance N°10/CMLN du 18 mars 1971 fixant le taux de la «taxe-logement» ;\n\nVu la Loi N°94-009 du 22 mars 1994 portant principes fondamentaux de la création, de l\x92organisation, de la gestion et\ndu contrôle des services publics ;\n\nVu la Loi N°90-110/AN-RM du 18 octobre 1990 portant principes fondamentaux de la création, de l\x92organisation et du\nfonctionnement des Etablissements Publics à caractère Administratif ;\n\nVu la Loi N°96-030 du 12 juin 1996 portant création de l\x92Office Malien de l\x92Habitat ;\n\nVu le Décr